In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision
from torchsummary import summary
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import numpy as np
import torch.nn.functional as F
from tqdm import tqdm
from dataset import MNIST_Moving

from encoder import Embedded_Encoder
from decoder import Embedded_Decoder
from conv_lstm import ConvLSTM
from model import Model

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [47]:
train_set = MNIST_Moving(root='.data/mnist', train=True, download=True)
test_set = MNIST_Moving(root='.data/mnist', train=False, download=True)

batch_size = 1

train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_size,
                 
                 shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test_set,
                batch_size=batch_size,
                shuffle=False)

In [6]:
input= next(iter(train_loader))
print("The shape of the input is ", input.shape)

vid_seq = input[0]
print("The shape of the video seq is", vid_seq.shape)



The shape of the input is  torch.Size([100, 20, 1, 64, 64])
The shape of the video seq is torch.Size([20, 1, 64, 64])


In [5]:
# ModelEncoder = Embedded_Encoder(device)
# print("The input is of the size",vid_seq.shape)

# encoderLayerOutputs=ModelEncoder(vid_seq.float().cuda())
# #These features or outputs from each encoder can be then passed on to the convlstm 

# print("The outputs of each encoder in the embedded encoder:")
# for ftr in encoderLayerOutputs: print(ftr.shape)


# ModelDecoder=Embedded_Decoder()
# print("The input of the embedded decoder is the output from the last encoder in the embedded encoder",encoderLayerOutputs[-1].shape)
# decoderLayerOutputs=ModelDecoder(encoderLayerOutputs[-1])
# print("The outputs of each encoder in the embedded decoder:")
# for ftr in decoderLayerOutputs: print(ftr.shape)

In [39]:
model = Model()
input_seq = torch.rand(((20, 1, 64, 64))).to(device)
model(input_seq).shape

torch.Size([20, 1, 64, 64])

In [7]:
abc = torch.rand((1, 64, 16, 16))
abc_3 = torch.rand((1, 256, 4, 4))

llm = conv_model_FirstEncoder(abc)
llm_3 = conv_model_ThirdEncoder(abc_3)

In [43]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.2)


In [44]:
# device = "cuda"
# def training(criterion):
# #     Seq of 20 images
#     input_seq = torch.rand(((20, 1, 64, 64)))
#     embd_model = Embedded_Encoder(device)
#     embds = embd_model(input_seq.float().cuda())
#     lstm_1, (h_1, c_1) = conv_model_FirstEncoder(embds[0])
#     lstm_2, (h_2, c_2) = conv_model_SecondEncoder(embds[1])
#     lstm_3, (h_3, c_3) = conv_model_ThirdEncoder(embds[2])
    
#     print(lstm_1.shape)
#     print(lstm_2.shape)
#     print(lstm_3.shape)
#     decoder = Embedded_Decoder(device)
#     out_1 = decoder.firstDecoder(lstm_3.squeeze())
#     print(out_1.shape)
#     in_2 = torch.cat((out_1, lstm_2.squeeze()), 1)
#     out_2 = decoder.secondDecoder(in_2)
#     print(out_2.shape)
#     in_3 = torch.cat((out_2, lstm_1.squeeze()), 1)
#     out_3 = decoder.thirdDecoder(in_3)
#     print(out_3.shape)
    
#     loss = criterion(input_seq[10:, :, :, :].to(device), out_3[10:, :, :, :])
#     print(loss)

In [51]:
def train_epoch(model, train_loader, optimizer, criterion, epoch, device):
    """ Training a model for one epoch """
    
    loss_list = []
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader))
    for i, full_seq in progress_bar:
       
        full_seq = full_seq.squeeze(0).type(torch.FloatTensor).to(device)
        target_seq = full_seq[10:, :, :, :]
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass only to get logits/output
        outputs = model(full_seq)
        outputs = outputs.to(device)
         
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs[10:, :, :, :], target_seq)
        loss_list.append(loss.item())
        
        # Getting gradients w.r.t. parameters
        loss.backward()
         
        # Updating parameters
        optimizer.step()
        
        progress_bar.set_description(f"Epoch {epoch+1} Iter {i+1}: loss {loss.item():.5f}. ")
        
    mean_loss = np.mean(loss_list)
    return mean_loss, loss_list

In [52]:
train_epoch(model, train_loader, optimizer, criterion, 0, device)

Epoch 1 Iter 439: loss 1147.03320. :   5%|▏   | 439/9000 [00:39<12:48, 11.15it/s]


KeyboardInterrupt: 

In [ ]:
decoder.firstDecoder

9000